# Capstone Project - Battle of the Neighbourhoods

### In this project we shall check house prices in Birmingham UK, using the national registry data. We shall then come up with various recommendations based on house pricing limits that the customer has given us and we shall recommend the best places to buy such a house in the Birmingham City area. The venues and the facilities around that area will be presented to the customer and they can decide if they wish to buy the property based on price, location based on postcode, venues such as restaurants and facilities such as supermarkets. Also our customer is worried about criminal activity and would like to know if there are any reported crimes in the areas that we propose to him. I also use a k-means clustering algorithm based on the facilities in each neighbourhood and present it to the customer to inform his decision about the property

In [1]:
#We start this project by importing all the libraries we shall be using first
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.05 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.61 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   2.87 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  21.33 MB/s
vincent-0.4.4- 100% |###################

In [2]:
#Let us read the csv file from landregistry.data.gov.uk that contains the information 
#about house pricing per street and postcode in England and Wales
url = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-monthly-update-new-version.csv'
eng_houses = pd.read_csv(url)
#We have put in a dataframe and now we want to read it.
eng_houses.head()

,{773788C2-3852-2CE4-E053-6C04A8C05E57},148950,2004-07-16 00:00,BS22 9HQ,T,N,F,1,Unnamed: 8,SAVERNAKE ROAD,Unnamed: 10,WESTON-SUPER-MARE,NORTH SOMERSET,NORTH SOMERSET.1,A,A.1
0,{773788C2-5C4A-2CE4-E053-6C04A8C05E57},230000,2004-11-05 00:00,PL34 0HB,S,N,F,GLEN HOUSE,1,NaN,TREBARWITH STRAND,TINTAGEL,CORNWALL,CORNWALL,A,A
1,{773788C2-6BBD-2CE4-E053-6C04A8C05E57},340000,2004-01-20 00:00,EX5 3DZ,D,N,F,2,NaN,SHERCROFT CLOSE,BROADCLYST,EXETER,EAST DEVON,DEVON,A,A
2,{773788C2-6BEC-2CE4-E053-6C04A8C05E57},175000,2004-06-04 00:00,EX38 8NL,D,N,F,LANE END,NaN,NaN,WITHACOTT,TORRINGTON,TORRIDGE,DEVON,A,A
3,{773788C2-71DA-2CE4-E053-6C04A8C05E57},190000,2004-01-30 00:00,DT2 7LF,F,N,L,4,NaN,BARTON FARM,CERNE ABBAS,DORCHESTER,WEST DORSET,DORSET,A,A
4,{773788C2-8177-2CE4-E053-6C04A8C05E57},163000,2004-01-26 00:00,N16 7LF,F,N,L,4,NaN,GARNHAM CLOSE,NaN,LONDON,HACKNEY,GREATER LONDON,A,A


In [3]:
# Assign proper column names to the file we downloaded
eng_houses.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [4]:
# Format the date column
eng_houses['Date_Transfer'] = eng_houses['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016 we dont care about them
eng_houses.drop(eng_houses[eng_houses.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
eng_houses.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [5]:
#Choose the entries for Birmingham UK only
eng_houses_birm = eng_houses.query("Town_City == 'BIRMINGHAM'")

# Make a list of post codes in Birmingham
postcodes = eng_houses_birm['Postcode'].unique().tolist()

In [6]:
#Show me the postcodes
postcodes

['B38 9RD',
 'B16 9LQ',
 'B31 2LT',
 'B11 2HR',
 'B24 9RN',
 'B32 2BX',
 'B23 5AB',
 'B33 9NH',
 'B45 9PD',
 'B48 7NE',
 'B14 6EZ',
 'B30 3QS',
 'B17 0JH',
 'B6 6LA',
 'B23 6EX',
 'B20 1LS',
 'B8 1HN',
 'B28 9PG',
 'B13 8JQ',
 'B44 8AY',
 'B44 0JY',
 'B26 2UX',
 'B31 1QD',
 'B20 2LX',
 'B21 9HA',
 'B27 6LA',
 'B45 8EH',
 'B5 7UN',
 'B36 0JQ',
 'B11 3LL',
 'B5 7TS',
 'B29 7RR',
 'B33 9LN',
 'B36 8JT',
 'B15 3TP',
 'B15 3RU',
 'B34 7RE',
 'B30 2XY',
 'B33 0TS',
 'B44 0BG',
 'B28 0JX',
 'B44 8RR',
 'B11 3DE',
 'B45 8SR',
 'B12 8AQ',
 'B34 6HA',
 'B36 9JB',
 'B8 2LS',
 'B36 0QE',
 'B43 6DY',
 'B20 2HX',
 'B24 8HE',
 'B29 7AF',
 'B21 8HB',
 'B28 0NG',
 'B29 4NS',
 'B27 6HY',
 'B24 9JY',
 'B8 2TR',
 'B27 6SR',
 'B13 0RU',
 'B21 0TR',
 'B15 3LA',
 'B14 7QN',
 'B26 3BB',
 'B13 9NU',
 'B5 7SY',
 'B34 6PH',
 'B43 7AP',
 'B35 6PE',
 'B5 7SN',
 'B14 4HJ',
 'B20 3BG',
 'B30 2DD',
 'B21 8JL',
 'B15 2EN',
 'B30 2BN',
 'B37 6UP',
 'B43 7AA',
 'B5 4TD',
 'B26 3BG',
 'B5 7UR',
 'B11 2HT',
 'B38 8YG',
 '

In [7]:
#Group houses by price and Postcode
eng_group_price1 = eng_houses_birm.groupby(['Postcode'])['Price'].mean().reset_index()

# Give meaningful names to the columns
eng_group_price1.columns = ['Postcode','Avg_Price']

In [8]:
#Group houses by price and Street Name
eng_group_price2 = eng_houses_birm.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
eng_group_price2.columns = ['Street','Avg_Price']

In [9]:
#Find housing prices between 400000 and 500000 - this is what the customer can afford
#using the wider post code
birm_can_buy1 = eng_group_price1.query("(Avg_Price >= 400000) & (Avg_Price <= 500000)")

In [29]:
# Display all the houses in the above price range using the postcode
birm_can_buy1

,Postcode,Avg_Price
78,B12 8LQ,475000.0
84,B12 9QA,400000.0
87,B13 0AJ,400000.0
88,B13 0AL,425000.0
157,B14 6BB,484000.0
159,B14 6DL,420500.0
175,B14 7DB,403850.0
181,B14 7NR,415000.0
204,B15 2LB,427500.0
206,B15 2NR,485000.0


In [11]:
#How many postcode areas we identified within this price range
len(birm_can_buy1)

40

In [12]:
#Find housing prices between 400000 and 500000 - this is what the customer can afford
#using the street name
birm_can_buy2 = eng_group_price2.query("(Avg_Price >= 400000) & (Avg_Price <= 500000)")

In [13]:
#Show me the result
birm_can_buy2

,Street,Avg_Price
51,ASTON CHURCH ROAD,435000.0
65,BARNT GREEN ROAD,420000.0
141,BROADSTONE ROAD,500000.0
171,CALLOW HILL ROAD,432500.0
258,COURT OAK ROAD,485000.0
266,CRONDAL PLACE,427500.0
285,DEELEY CLOSE,485000.0
325,EDGBASTON ROAD,400000.0
335,ELMDON LANE,480000.0
352,FEATHERSTONE ROAD,484000.0


In [14]:
# The code was removed by Watson Studio for sharing.

,Postcode,Latitude,Longitude,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,B1 1AA,52.476660,-1.903535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B1 1AD,52.476660,-1.903535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B1 1AG,52.474528,-1.902156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B1 1AH,52.476391,-1.904257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B1 1AQ,52.474528,-1.902156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#I have inserted a file from my IBM data storage the csv with the postcodes and the lat and long
#but because the entry contains credentials it must be a hidden cell

In [16]:
#Make the dataset look nice and tidy
new_birm_data=birm_data.iloc[:, 0:3]

In [17]:
#Now it looks nice and tidy!
new_birm_data

,Postcode,Latitude,Longitude
0,B1 1AA,52.476660,-1.903535
1,B1 1AD,52.476660,-1.903535
2,B1 1AG,52.474528,-1.902156
3,B1 1AH,52.476391,-1.904257
4,B1 1AQ,52.474528,-1.902156
5,B1 1AT,52.476391,-1.904257
6,B1 1AY,52.475924,-1.905392
7,B1 1AZ,52.476660,-1.903535
8,B1 1BA,52.475425,-1.900211
9,B1 1BB,52.480783,-1.904096


In [43]:
#I have downloaded a csv file with all the Birmingham post codes and I have uploaded it 
#on to ibm watson data store. I have used the insert to code tool to add it to my code
#as it uses credentials it is a hidden cell
#From the website https://www.freemaptools.com/download-uk-postcode-lat-lng.htm

In [37]:
#Now let us merge the two dataframes - the prices we found and the latitude and longitute

In [18]:
birm_loc = pd.merge(birm_can_buy1, new_birm_data, on = ['Postcode'], how='inner')

In [19]:
#Now let us see the dataframe that we have created
birm_loc.head(10)

,Postcode,Avg_Price,Latitude,Longitude
0,B12 8LQ,475000.0,52.457360,-1.876749
1,B12 9QA,400000.0,52.453640,-1.897434
2,B13 0AJ,400000.0,52.434642,-1.867928
3,B13 0AL,425000.0,52.434262,-1.865488
4,B14 6BB,484000.0,52.425418,-1.894720
5,B14 6DL,420500.0,52.421608,-1.896597
6,B14 7DB,403850.0,52.434463,-1.890192
7,B14 7NR,415000.0,52.427775,-1.907745
8,B15 2LB,427500.0,52.467049,-1.914994
9,B15 2NR,485000.0,52.466535,-1.912360


In [20]:
#Let us find the coordinates for Birmingham
address = 'Birmingham, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Birmingham are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Birmingham are 52.4775396, -1.894053.


In [21]:
# create map of Birmingham  using latitude and longitude values for the price range we have identified
map_birm = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(birm_loc['Latitude'], birm_loc['Longitude'], birm_loc['Avg_Price'], birm_loc['Postcode']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_birm)  
    
map_birm

In [22]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: MJUJMLKUGZZTSBACKPUGEAELZ2GQCQNDJQRDAFYU0WCXDOQD
CLIENT_SECRET:HEQLCBQSFCX5RVSP4KBFXJS33S0ND5QBECBZ3FYDP2HUFEKW


In [23]:
#Please note that I have hidden the actual code with my Foursquare credentials
#This is an example of what I did for your reference
#CLIENT_ID = '******' # your Foursquare ID
#CLIENT_SECRET = '*******' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [24]:
#lets get the foursquare venues around these postcodes
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
#Get the postcodes and the venues
location_venues = getNearbyVenues(names=birm_loc['Postcode'],
                                   latitudes=birm_loc['Latitude'],
                                   longitudes=birm_loc['Longitude']
                                  )

B12 8LQ
B12 9QA
B13 0AJ
B13 0AL
B14 6BB
B14 6DL
B14 7DB
B14 7NR
B15 2LB
B15 2NR
B15 2UU
B15 3NG
B17 0AQ
B17 0TD
B17 8NR
B17 9HG
B17 9SY
B18 6BS
B24 0BD
B26 2BN
B28 0JX
B28 8DQ
B28 9BQ
B29 7DH
B30 1AB
B31 2FW
B31 2GH
B32 2ED
B37 7DN
B45 8GQ
B45 8JZ
B45 8NW
B45 8PJ
B45 8PR
B47 6AQ
B47 6AW
B47 6AX
B48 7LS
B48 7QL
B7 5RX


In [26]:
#Show the venues around the postcodes that we have identified
location_venues

,Postcode,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,B12 8LQ,52.457360,-1.876749,Lahore,52.456860,-1.876458,Indian Restaurant
1,B12 8LQ,52.457360,-1.876749,Al-Frash Balti,52.457261,-1.876494,Indian Restaurant
2,B12 8LQ,52.457360,-1.876749,Bader Restaurant,52.457440,-1.876448,Moroccan Restaurant
3,B12 8LQ,52.457360,-1.876749,Pepe's Piri Piri,52.454712,-1.877462,Fast Food Restaurant
4,B12 8LQ,52.457360,-1.876749,Diwan Balti,52.453767,-1.887040,Indian Restaurant
5,B12 8LQ,52.457360,-1.876749,Antep Turkish Cuisine,52.453313,-1.877929,Turkish Restaurant
6,B12 8LQ,52.457360,-1.876749,Imrans,52.455440,-1.876861,Indian Restaurant
7,B12 8LQ,52.457360,-1.876749,Heavenly Desserts,52.452298,-1.878734,Dessert Shop
8,B12 8LQ,52.457360,-1.876749,Lidl,52.455569,-1.886675,Supermarket
9,B12 8LQ,52.457360,-1.876749,Shabab,52.457450,-1.876531,Pakistani Restaurant


In [27]:
#Group the venues by postcode and count them
birm_venues=location_venues.groupby('Postcode').count()

In [28]:
#Show me the type of the structure I have created
type(birm_venues)

pandas.core.frame.DataFrame

In [29]:
#Print it now
birm_venues

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
B12 8LQ,24,24,24,24,24,24
B12 9QA,39,39,39,39,39,39
B13 0AJ,12,12,12,12,12,12
B13 0AL,11,11,11,11,11,11
B14 6BB,19,19,19,19,19,19
B14 6DL,9,9,9,9,9,9
B14 7DB,26,26,26,26,26,26
B14 7NR,12,12,12,12,12,12
B15 2LB,58,58,58,58,58,58


In [30]:
#Now get the nearby facilities in a radius of 5000 and the limit being 100 entries
def getNearbyFacility(names, latitudes, longitudes, radius=5000):
    
    facility_list=[]
    
    radius = 5000
    LIMIT = 100
    categories = '4bf58dd8d48988d196941735,58daa1558bbb0b01f18ec1f7,4bf58dd8d48988d13d941735,4f4533804b9074f6e4fb0105,4bf58dd8d48988d118951735'
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        categories,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

        # make the GET request
        results = requests.get(url).json()['response']

        # return only relevant information for each nearby venue
        facility_list.append([(
            name,
            facility['name'],
            facility['categories'][0]['name'],
            facility['location']['distance'], 
            facility['location']['lat'],
            facility['location']['lng']) for facility in results['venues']])

    nearby_facility = pd.DataFrame([item for f_list in facility_list for item in f_list])
    nearby_facility.columns = ['Postcode',
                  'Facility Name',             
                  'Facility Category', 
                  'Distance', 
                  'Facility Latitude', 
                  'Facility Longitude']
    
    return(nearby_facility)

In [31]:
#Get the locations for the facilities near these postcodes
location_facility = getNearbyFacility(names=birm_loc['Postcode'],
                                      latitudes=birm_loc['Latitude'],
                                      longitudes=birm_loc['Longitude']
                                  )

B12 8LQ
B12 9QA
B13 0AJ
B13 0AL
B14 6BB
B14 6DL
B14 7DB
B14 7NR
B15 2LB
B15 2NR
B15 2UU
B15 3NG
B17 0AQ
B17 0TD
B17 8NR
B17 9HG
B17 9SY
B18 6BS
B24 0BD
B26 2BN
B28 0JX
B28 8DQ
B28 9BQ
B29 7DH
B30 1AB
B31 2FW
B31 2GH
B32 2ED
B37 7DN
B45 8GQ
B45 8JZ
B45 8NW
B45 8PJ
B45 8PR
B47 6AQ
B47 6AW
B47 6AX
B48 7LS
B48 7QL
B7 5RX


In [32]:
#Show them to us
location_facility

,Postcode,Facility Name,Facility Category,Distance,Facility Latitude,Facility Longitude
0,B12 8LQ,Aldi,Supermarket,394,52.459437,-1.872041
1,B12 8LQ,Tesco,Supermarket,4272,52.485462,-1.919668
2,B12 8LQ,Birmingham Heartlands Hospital,Medical Center,4001,52.478744,-1.829317
3,B12 8LQ,Tesco,Grocery Store,2679,52.473007,-1.906775
4,B12 8LQ,Morrisons,Supermarket,3439,52.471578,-1.921778
5,B12 8LQ,Aldi,Supermarket,4210,52.444682,-1.935228
6,B12 8LQ,Tesco,Grocery Store,2877,52.479293,-1.899214
7,B12 8LQ,Tesco,Grocery Store,3840,52.446681,-1.930574
8,B12 8LQ,Queen Elizabeth Hospital (QEHB),Hospital,4444,52.451361,-1.941522
9,B12 8LQ,Aldi,Supermarket,1962,52.454670,-1.905334


In [33]:
#Group the facilities by postcode
location_facility.groupby('Postcode').count()

,Facility Name,Facility Category,Distance,Facility Latitude,Facility Longitude
Postcode,,,,,
B12 8LQ,50,50,50,50,50
B12 9QA,50,50,50,50,50
B13 0AJ,50,50,50,50,50
B13 0AL,50,50,50,50,50
B14 6BB,50,50,50,50,50
B14 6DL,50,50,50,50,50
B14 7DB,50,50,50,50,50
B14 7NR,50,50,50,50,50
B15 2LB,50,50,50,50,50


In [34]:
#See how many unique categories exist in each post code area
print('There are {} unique categories in this Birmingham Post Code area.'.format(len(location_venues['Venue Category'].unique())))

There are 129 unique categories in this Birmingham Post Code area.


In [35]:
#Analyse each neighbourhood
# one hot encoding
birm_onehot = pd.get_dummies(location_facility[['Facility Category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
birm_onehot['Postcode'] = location_facility['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [birm_onehot.columns[-1]] + list(birm_onehot.columns[:-1])
birm_onehot = birm_onehot[fixed_columns]
birm_onehot

,Postcode,Convenience Store,Dentist's Office,Elementary School,Food & Drink Shop,Gas Station,Grocery Store,High School,Hospital,Hospital Ward,Medical Center,Middle School,School,Supermarket
0,B12 8LQ,0,0,0,0,0,0,0,0,0,0,0,0,1
1,B12 8LQ,0,0,0,0,0,0,0,0,0,0,0,0,1
2,B12 8LQ,0,0,0,0,0,0,0,0,0,1,0,0,0
3,B12 8LQ,0,0,0,0,0,1,0,0,0,0,0,0,0
4,B12 8LQ,0,0,0,0,0,0,0,0,0,0,0,0,1
5,B12 8LQ,0,0,0,0,0,0,0,0,0,0,0,0,1
6,B12 8LQ,0,0,0,0,0,1,0,0,0,0,0,0,0
7,B12 8LQ,0,0,0,0,0,1,0,0,0,0,0,0,0
8,B12 8LQ,0,0,0,0,0,0,0,1,0,0,0,0,0
9,B12 8LQ,0,0,0,0,0,0,0,0,0,0,0,0,1


In [36]:
#Lets group them by postcode
birm_grouped = birm_onehot.groupby('Postcode').mean().reset_index()
birm_grouped

,Postcode,Convenience Store,Dentist's Office,Elementary School,Food & Drink Shop,Gas Station,Grocery Store,High School,Hospital,Hospital Ward,Medical Center,Middle School,School,Supermarket
0,B12 8LQ,0.000000,0.00,0.000000,0.000000,0.000000,0.500000,0.020000,0.200000,0.00,0.02,0.00,0.000000,0.260000
1,B12 9QA,0.000000,0.00,0.040000,0.000000,0.000000,0.520000,0.020000,0.200000,0.00,0.00,0.00,0.000000,0.220000
2,B13 0AJ,0.020000,0.02,0.040000,0.000000,0.000000,0.560000,0.020000,0.080000,0.00,0.02,0.00,0.000000,0.240000
3,B13 0AL,0.020000,0.02,0.040000,0.000000,0.000000,0.580000,0.020000,0.080000,0.00,0.00,0.00,0.000000,0.240000
4,B14 6BB,0.020000,0.02,0.060000,0.000000,0.020000,0.460000,0.020000,0.160000,0.00,0.02,0.00,0.000000,0.220000
5,B14 6DL,0.020000,0.02,0.060000,0.000000,0.020000,0.460000,0.040000,0.160000,0.00,0.02,0.00,0.000000,0.200000
6,B14 7DB,0.020000,0.00,0.040000,0.000000,0.000000,0.500000,0.020000,0.200000,0.00,0.00,0.00,0.000000,0.220000
7,B14 7NR,0.020000,0.02,0.060000,0.000000,0.020000,0.420000,0.060000,0.200000,0.00,0.02,0.00,0.000000,0.180000
8,B15 2LB,0.000000,0.00,0.040000,0.000000,0.000000,0.540000,0.020000,0.180000,0.00,0.00,0.00,0.000000,0.220000
9,B15 2NR,0.000000,0.00,0.040000,0.000000,0.000000,0.520000,0.020000,0.200000,0.00,0.00,0.00,0.000000,0.220000


In [37]:
#Lets check the shape of the array
birm_grouped.shape

(40, 14)

In [38]:
#Lets print the most common venues
num_top_venues = 5

for hood in birm_grouped['Postcode']:
    print("----"+hood+"----")
    temp = birm_grouped[birm_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----B12 8LQ----
            venue  freq
0   Grocery Store  0.50
1     Supermarket  0.26
2        Hospital  0.20
3     High School  0.02
4  Medical Center  0.02


----B12 9QA----
               venue  freq
0      Grocery Store  0.52
1        Supermarket  0.22
2           Hospital  0.20
3  Elementary School  0.04
4        High School  0.02


----B13 0AJ----
               venue  freq
0      Grocery Store  0.56
1        Supermarket  0.24
2           Hospital  0.08
3  Elementary School  0.04
4  Convenience Store  0.02


----B13 0AL----
               venue  freq
0      Grocery Store  0.58
1        Supermarket  0.24
2           Hospital  0.08
3  Elementary School  0.04
4  Convenience Store  0.02


----B14 6BB----
               venue  freq
0      Grocery Store  0.46
1        Supermarket  0.22
2           Hospital  0.16
3  Elementary School  0.06
4  Convenience Store  0.02


----B14 6DL----
               venue  freq
0      Grocery Store  0.46
1        Supermarket  0.20
2           Hospital 

In [39]:
#Lets put that in a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Facility'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Facility'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = birm_grouped['Postcode']

for ind in np.arange(birm_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(birm_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Postcode,1st Most Common Facility,2nd Most Common Facility,3rd Most Common Facility,4th Most Common Facility,5th Most Common Facility,6th Most Common Facility,7th Most Common Facility,8th Most Common Facility,9th Most Common Facility,10th Most Common Facility
0,B12 8LQ,Grocery Store,Supermarket,Hospital,Medical Center,High School,School,Middle School,Hospital Ward,Gas Station,Food & Drink Shop
1,B12 9QA,Grocery Store,Supermarket,Hospital,Elementary School,High School,School,Middle School,Medical Center,Hospital Ward,Gas Station
2,B13 0AJ,Grocery Store,Supermarket,Hospital,Elementary School,Medical Center,High School,Dentist's Office,Convenience Store,School,Middle School
3,B13 0AL,Grocery Store,Supermarket,Hospital,Elementary School,High School,Dentist's Office,Convenience Store,School,Middle School,Medical Center
4,B14 6BB,Grocery Store,Supermarket,Hospital,Elementary School,Medical Center,High School,Gas Station,Dentist's Office,Convenience Store,School
5,B14 6DL,Grocery Store,Supermarket,Hospital,Elementary School,High School,Medical Center,Gas Station,Dentist's Office,Convenience Store,School
6,B14 7DB,Grocery Store,Supermarket,Hospital,Elementary School,High School,Convenience Store,School,Middle School,Medical Center,Hospital Ward
7,B14 7NR,Grocery Store,Hospital,Supermarket,High School,Elementary School,Medical Center,Gas Station,Dentist's Office,Convenience Store,School
8,B15 2LB,Grocery Store,Supermarket,Hospital,Elementary School,High School,School,Middle School,Medical Center,Hospital Ward,Gas Station
9,B15 2NR,Grocery Store,Supermarket,Hospital,Elementary School,High School,School,Middle School,Medical Center,Hospital Ward,Gas Station


In [41]:
# set number of clusters
kclusters = 5

birm_grouped_clustering = birm_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(birm_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [42]:
new_birm_data.head(5)

,Postcode,Latitude,Longitude
0,B1 1AA,52.476660,-1.903535
1,B1 1AD,52.476660,-1.903535
2,B1 1AG,52.474528,-1.902156
3,B1 1AH,52.476391,-1.904257
4,B1 1AQ,52.474528,-1.902156


In [43]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each Postcode
birm_merged1 = neighborhoods_venues_sorted

# add clustering labels
birm_merged1['Cluster Labels'] = kmeans.labels_
birm_merged1

,Postcode,1st Most Common Facility,2nd Most Common Facility,3rd Most Common Facility,4th Most Common Facility,5th Most Common Facility,6th Most Common Facility,7th Most Common Facility,8th Most Common Facility,9th Most Common Facility,10th Most Common Facility,Cluster Labels
0,B12 8LQ,Grocery Store,Supermarket,Hospital,Medical Center,High School,School,Middle School,Hospital Ward,Gas Station,Food & Drink Shop,0
1,B12 9QA,Grocery Store,Supermarket,Hospital,Elementary School,High School,School,Middle School,Medical Center,Hospital Ward,Gas Station,0
2,B13 0AJ,Grocery Store,Supermarket,Hospital,Elementary School,Medical Center,High School,Dentist's Office,Convenience Store,School,Middle School,0
3,B13 0AL,Grocery Store,Supermarket,Hospital,Elementary School,High School,Dentist's Office,Convenience Store,School,Middle School,Medical Center,0
4,B14 6BB,Grocery Store,Supermarket,Hospital,Elementary School,Medical Center,High School,Gas Station,Dentist's Office,Convenience Store,School,0
5,B14 6DL,Grocery Store,Supermarket,Hospital,Elementary School,High School,Medical Center,Gas Station,Dentist's Office,Convenience Store,School,0
6,B14 7DB,Grocery Store,Supermarket,Hospital,Elementary School,High School,Convenience Store,School,Middle School,Medical Center,Hospital Ward,0
7,B14 7NR,Grocery Store,Hospital,Supermarket,High School,Elementary School,Medical Center,Gas Station,Dentist's Office,Convenience Store,School,0
8,B15 2LB,Grocery Store,Supermarket,Hospital,Elementary School,High School,School,Middle School,Medical Center,Hospital Ward,Gas Station,0
9,B15 2NR,Grocery Store,Supermarket,Hospital,Elementary School,High School,School,Middle School,Medical Center,Hospital Ward,Gas Station,0


In [45]:
#Letus put everything together and add the Lat and Long
birm_merged2=pd.merge(new_birm_data, birm_merged1, on='Postcode')

In [46]:
type(birm_merged2)

pandas.core.frame.DataFrame

In [47]:
birm_merged2

,Postcode,Latitude,Longitude,1st Most Common Facility,2nd Most Common Facility,3rd Most Common Facility,4th Most Common Facility,5th Most Common Facility,6th Most Common Facility,7th Most Common Facility,8th Most Common Facility,9th Most Common Facility,10th Most Common Facility,Cluster Labels
0,B12 8LQ,52.457360,-1.876749,Grocery Store,Supermarket,Hospital,Medical Center,High School,School,Middle School,Hospital Ward,Gas Station,Food & Drink Shop,0
1,B12 9QA,52.453640,-1.897434,Grocery Store,Supermarket,Hospital,Elementary School,High School,School,Middle School,Medical Center,Hospital Ward,Gas Station,0
2,B13 0AJ,52.434642,-1.867928,Grocery Store,Supermarket,Hospital,Elementary School,Medical Center,High School,Dentist's Office,Convenience Store,School,Middle School,0
3,B13 0AL,52.434262,-1.865488,Grocery Store,Supermarket,Hospital,Elementary School,High School,Dentist's Office,Convenience Store,School,Middle School,Medical Center,0
4,B14 6BB,52.425418,-1.894720,Grocery Store,Supermarket,Hospital,Elementary School,Medical Center,High School,Gas Station,Dentist's Office,Convenience Store,School,0
5,B14 6DL,52.421608,-1.896597,Grocery Store,Supermarket,Hospital,Elementary School,High School,Medical Center,Gas Station,Dentist's Office,Convenience Store,School,0
6,B14 7DB,52.434463,-1.890192,Grocery Store,Supermarket,Hospital,Elementary School,High School,Convenience Store,School,Middle School,Medical Center,Hospital Ward,0
7,B14 7NR,52.427775,-1.907745,Grocery Store,Hospital,Supermarket,High School,Elementary School,Medical Center,Gas Station,Dentist's Office,Convenience Store,School,0
8,B15 2LB,52.467049,-1.914994,Grocery Store,Supermarket,Hospital,Elementary School,High School,School,Middle School,Medical Center,Hospital Ward,Gas Station,0
9,B15 2NR,52.466535,-1.912360,Grocery Store,Supermarket,Hospital,Elementary School,High School,School,Middle School,Medical Center,Hospital Ward,Gas Station,0


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(birm_merged2['Latitude'], birm_merged2['Longitude'], birm_merged2['Postcode'], birm_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
# The code was removed by Watson Studio for sharing.

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,4c72fc691d47a7fc5b9bfb8dea15118c655de8562c8741...,2018-09,West Midlands Police,West Midlands Police,-1.850056,52.591108,On or near Walsall Road,E01009417,Birmingham 001A,Other theft,Under investigation,NaN
1,b9268550deecebc3fb320d328b3ca456fd908c74e33842...,2018-09,West Midlands Police,West Midlands Police,-1.843740,52.594588,On or near Clarence Road,E01009418,Birmingham 001B,Burglary,Investigation complete; no suspect identified,NaN
2,59a2818666537fab8b78487798c0274781a0a382906bcb...,2018-09,West Midlands Police,West Midlands Police,-1.842882,52.599289,On or near Balmoral Road,E01009418,Birmingham 001B,Burglary,Investigation complete; no suspect identified,NaN
3,2bb68537983ed3f86548cd523a2989e0ab881305d9b57b...,2018-09,West Midlands Police,West Midlands Police,-1.845780,52.593827,On or near Hook Drive,E01009418,Birmingham 001B,Other theft,Under investigation,NaN
4,36b42dd91a71e83e763fb55300f50d8a961056c97d71c2...,2018-09,West Midlands Police,West Midlands Police,-1.839063,52.597809,On or near Byron Court,E01009418,Birmingham 001B,Violence and sexual offences,Unable to prosecute suspect,NaN


In [51]:
#I have inserted in my IBM object store the file I downloaded from the official UK police website with the crime data
#http://www.ukcrimestats.com/Police_Force/West_Midlands_Police#

In [52]:
#Let us see how many crimes by lat and long in Birmingham are reported
#birm_crime_data.groupby(['Longitude','Latitude']).size()
#birm_crime_data({'count' : birm_crime_data.groupby( [ "Latitude", "Longitude"] ).size()}).reset_index()
crime_lat_lon=birm_crime_data.groupby( [ "Latitude", "Longitude"] ).size().to_frame(name = 'Crimes').reset_index()

In [53]:
#Display locations and count of crimes in all Birmingham Post codes
crime_lat_lon

,Latitude,Longitude,Crimes
0,50.206416,-3.718838,1
1,50.606908,-1.948141,1
2,50.607538,-1.983865,1
3,50.613688,-2.499717,1
4,50.704360,-1.913835,2
5,50.706596,-1.909554,1
6,50.718761,-3.528194,4
7,50.719380,-1.788344,1
8,50.721637,-1.832744,1
9,50.775585,-3.143992,3


In [54]:
locations = crime_lat_lon[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)

11301

In [55]:
locationlist

[[50.206416, -3.718838],
 [50.606908000000004, -1.948141],
 [50.607538, -1.9838650000000002],
 [50.613688, -2.499717],
 [50.70436, -1.913835],
 [50.706596000000005, -1.9095540000000002],
 [50.718761, -3.528194],
 [50.71938, -1.788344],
 [50.721637, -1.832744],
 [50.775585, -3.143992],
 [50.784687, -2.0074099999999997],
 [50.786241, -1.9795770000000001],
 [50.797402000000005, -1.9977200000000002],
 [51.753, -1.9381529999999998],
 [51.784411, -0.17893900000000001],
 [52.29333, -1.960118],
 [52.349948, -1.7786560000000002],
 [52.353256, -1.778302],
 [52.354574, -1.6959119999999999],
 [52.355333, -1.697566],
 [52.355557, -1.773563],
 [52.356970000000004, -1.697673],
 [52.3585, -1.7800669999999998],
 [52.364004, -1.780995],
 [52.367102, -1.821632],
 [52.367616999999996, -1.8355970000000001],
 [52.369229, -1.758747],
 [52.369818, -1.514461],
 [52.369911, -1.7627529999999998],
 [52.370616999999996, -1.760575],
 [52.370675, -1.7580189999999998],
 [52.371072, -1.749909],
 [52.372006, -1.7962900

In [56]:
type(crime_lat_lon)

pandas.core.frame.DataFrame

In [57]:
#Merge the two datafarames to see if there are any reported crimes in the areas
#We have identified for our customer
birm_full = pd.merge(birm_loc,crime_lat_lon, how='left', left_on=['Latitude','Longitude'], right_on = ['Latitude','Longitude'])

In [58]:
birm_full
#Our Customer is really happy as in the selected areas there have been 
#no recent reported crimes

,Postcode,Avg_Price,Latitude,Longitude,Crimes
0,B12 8LQ,475000.0,52.457360,-1.876749,NaN
1,B12 9QA,400000.0,52.453640,-1.897434,NaN
2,B13 0AJ,400000.0,52.434642,-1.867928,NaN
3,B13 0AL,425000.0,52.434262,-1.865488,NaN
4,B14 6BB,484000.0,52.425418,-1.894720,NaN
5,B14 6DL,420500.0,52.421608,-1.896597,NaN
6,B14 7DB,403850.0,52.434463,-1.890192,NaN
7,B14 7NR,415000.0,52.427775,-1.907745,NaN
8,B15 2LB,427500.0,52.467049,-1.914994,NaN
9,B15 2NR,485000.0,52.466535,-1.912360,NaN
